
<img src="https://minas.medellin.unal.edu.co/eventos/latwaves2018/images/logo_unal.png" alt="unal logo" width="200"/>

# ML 2022-1 Final Project
## Introductory Programming Task Repair using CodeBERT
### Dataset builder
Date: 03/07/22

By 
 - Diego Alejandro Herrera daherrerabu@unal.edu.co
 - Jheison Alejandro Morales jhmoralesva@unal.edu.co


## Goal

- Build a dataset with the incorrect and correct submissions from the programming exercises. Using only introductory programming courses with Python, including the description and tests from INGINious server file system.

Pymongo offers us a simple interface to query the database using Python.

In [ ]:
##Optional pre-installations on Linux environments
!pip install pymongo

In [ ]:
from pymongo import MongoClient

In [ ]:
# conection string for localhost.
DB_HOST = "localhost"
DB_USERNAME = "root"
DB_PASSWORD = "20221"
DB_NAME = "INGInious"

CONNECTION_STRING = f"mongodb://{DB_USERNAME}:{DB_PASSWORD}@{DB_HOST}:27017/"

In [ ]:
# Create a connection using MongoClient.
client = MongoClient(CONNECTION_STRING)

In [ ]:
db = client[DB_NAME]

In [ ]:
list_collection_names = db.list_collection_names()
print(f"The database has {len(list_collection_names)} collections")
print(list_collection_names)

The database has 19 collections
['fs.chunks', 'services', 'users', 'storage', 'problem_banks', 'sessions', 'lis_outcome_queue', 'nonce', 'tasks_cache', 'user_hints', 'user_tasks', 'custom_tests', 'batch_jobs', 'users_changes', 'analytics', 'fs.files', 'plagiarism_checks', 'submissions', 'aggregations']


In [ ]:
import gridfs
import bson

fs = gridfs.GridFS(db)
fs_chunks_col = db["fs.chunks"]
fs_files_col = db["fs.files"]

fs_submissions_col = db["submissions"]
user_tasks_col = db['user_tasks']
problem_banks_col = db['problem_banks']
tasks_cache_col = db['tasks_cache']

In [ ]:
import pandas as pd
import os
import logging
logging.basicConfig(filename='dataset_builder.log', level=logging.DEBUG)

In [ ]:
logging.info('test')

In [ ]:
all_courses = fs_submissions_col.distinct("courseid")
all_courses_df = pd.DataFrame({'courseid': all_courses})

In [ ]:
all_courses_df['date'] = all_courses_df['courseid'].str.extract(r'(?P<date>\d{4}\-\d{1})')['date']
all_courses_df['group'] = all_courses_df['courseid'].str.extract(r'(?P<group>(?<=(G|g)roup)(\d+|\w+|\-\d+))')['group']
all_courses_df['name'] = all_courses_df['courseid'].str.extract(r'(?P<name>^\w+)')['name']

In [ ]:
all_courses_df.head()

,courseid,date,group,name
0,AYVDDCP-GroupMLDS-2021-2,2021-2,MLDS,AYVDDCP
1,AYVDDCP-GroupMLDS-2022-1,2022-1,MLDS,AYVDDCP
2,Algoritmos-Group1-2021-1,2021-1,1,Algoritmos
3,Algoritmos-Group1-2021-2,2021-2,1,Algoritmos
4,Algoritmos-Group2-2021-1,2021-1,2,Algoritmos


In [ ]:
all_courses_df.shape

(104, 4)

In [ ]:
all_courses_df['name'].unique()

array(['AYVDDCP', 'Algoritmos', 'BD', 'ED', 'EDD', 'IALCDLCYALP',
       'IALCDLCYLP', 'IALPCP', 'IALPCP2', 'IALSI', 'IAMLYS', 'ICCP',
       'IML', 'ISI', 'LDP', 'LP', 'ML', 'MLDS', 'MN', 'Maratones', 'PC',
       'PDC', 'PG', 'POO', 'QCP', 'QuantComp', 'TALP', 'TC', 'algoritmos',
       'ed1', 'grader', 'group', 'pc', 'poo', 'testing', 'toRemove'],
      dtype=object)

In [ ]:
all_courses_df['group'].unique()

array(['MLDS', '1', '2', '-1', '-2', '-5', '-3', '-4', '3', '4', '5', nan,
       '12', '17', '7', 's920', '11', '18', '22', '8', '-6', '15', '16',
       '6', '9'], dtype=object)

In [ ]:
courses_to_select = ['IALPCP', 'IALPCP2']
#courses_to_ignore = ['testing', 'toRemove', 'group', 'grader', 'QuantComp', 'ML', 'MLDS', 'TALP', 'Maratones', 'QCP', 'ISI', 'BD', 'PG', 'MN', 'ICCP', 'IML', 'AYVDDCP', 'IALCDLCYALP', 'IALCDLCYLP', 'IALSI', 'MLDS']
#groups_to_ignore = ['MLDS', 's920']

In [ ]:
all_courses_df = all_courses_df[all_courses_df['name'].isin(courses_to_select)]

In [ ]:
all_courses_df['name'] = all_courses_df['name'].str.lower()

/tmp/ipykernel_1096/1068694357.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_courses_df['name'] = all_courses_df['name'].str.lower()


In [ ]:
all_courses_df['name'].unique()

array(['ialpcp', 'ialpcp2'], dtype=object)

In [ ]:
all_courses_df.head()

,courseid,date,group,name
24,IALPCP-GroupMLDS-2021-2,2021-2,MLDS,ialpcp
25,IALPCP2-GroupMLDS-2021-2,2021-2,MLDS,ialpcp2


In [ ]:
all_courses_df.shape

(2, 4)

In [ ]:
def get_content(content):
    '''
    check if is a file or submited directly to platform as plain text.
    '''
    dtype = type(content)
    if  dtype == str:
        return content
    elif dtype == dict:
        return content['value']
    else:
        logging.warning(f"The data type of 'content': {dtype}, it is not the expected one, 'None' is returned. 'content': {content}")
        return None

In [ ]:
def get_lang_and_content_long_way(bson_dict):
    # default values
    dtype = 'Unknown'
    lang = 'Error'
    content = None
    k = [key for key in bson_dict.keys() if '/' not in key and '@' not in key]
    if len(k)==1:
        k = k[0]
        try:
            dtype = bson_dict[f'{k}/type']
            lang = bson_dict[f'{k}/language']
            content= get_content(bson_dict[k])
        except KeyError:
            logging.error(f"Can't recover lang or content using from keys: {k}")
    else:
        logging.error(f"Can't recover lang or content using from keys: {bson_dict.keys()}")
    return dtype, lang, content


In [ ]:
def get_lang_and_content(bson_dict):
    try:
        type = bson_dict['1/type']
        lang = bson_dict['1/language']
        
        content= get_content(bson_dict['1'])
    except KeyError:
        # try to recover
        return get_lang_and_content_long_way(bson_dict)
    return (type, lang, content)

In [ ]:
ext_dic = {
    'cpp': 'cpp',
    'cpp11': 'cpp',
    'java7': 'java',
    'python3': 'py',
    'verilog': 'v',
    'vhdl': 'vhdl',
    'python2': 'py',
    'java8': 'java',
    'c': 'c',
    'c11': 'c',
    'python': 'py',
    'notebook_file': 'ipynb'
}

def get_ext_by_lang(lang):
    try:
        ext = ext_dic[lang]
    except KeyError:
        ext = 'unk'
    return ext

In [ ]:
import hashlib

In [ ]:
a = 'Hello World'
hashlib.md5(a.encode()).hexdigest()

'b10a8db164e0754105b7a99be72e3fe5'

In [ ]:
def save_submissions_by_course(base_path, course_id, limit=0):
    '''
    base_path:
    course_id: 
    expected_lang: 
    limit:
    '''
    #lists
    tasks_list = []
    users_list = []
    results_list = []
    subs_list = []
    dates_list = []
    saved_list = []
    langs_list = []
    types_list = []
    
    # get the tasks
    tasks =  tasks_cache_col.find({'course_id': course_id}, limit=limit)
    for task in tasks:
        logging.info(f"Getting trials for courseid: {course_id}, taskid: {task['task_id']}")
        # get the students
        user_tasks_several_trials = user_tasks_col.find({'taskid': task['task_id'], 'grade': 100.0, 'tried': { '$gt':  1} }, limit=limit)
        for ut in user_tasks_several_trials:
            # only save data of users who have failed attempts
            failed_subs = fs_submissions_col.find({'taskid': task['task_id'], 'username': [ut['username']], 'status': 'done', 'result': 'failed'}).count()
            if(failed_subs == 0):
                continue
            hash_username = hashlib.md5(ut['username'].encode()).hexdigest()
            path = f"{base_path}/{task['task_id']}/{hash_username}"
            try:
                os.makedirs(f"{path}/success")
                os.makedirs(f"{path}/failed")
            except FileExistsError:
                logging.warning(f"Path: {path}/success or {path}/failed exist")

            # get the students submissions
            for sub in fs_submissions_col.find({'taskid': task['task_id'], 'username': [ut['username']], 'status': 'done'}):
                path_file = f"{path}/{sub['result']}/{sub['input']}"

                bson_dict = bson.BSON.decode(fs.get(sub['input']).read())
                
                dtype, lang, content = get_lang_and_content(bson_dict)

                # save in list
                tasks_list.append(task['task_id'])
                users_list.append(hash_username)
                results_list.append(sub['result'])
                subs_list.append(sub['input'])
                dates_list.append(sub['submitted_on'])
                langs_list.append(lang)
                types_list.append(dtype)

                if content != None:
                    
                    saved_list.append(True)
                    if dtype == 'code_multiple_languages':
                        # save code snippet
                        with open(f"{path_file}.{get_ext_by_lang(lang)}", "w") as f:
                            f.write(content)
                    else:
                        # save file
                        with open(f"{path_file}.{get_ext_by_lang(dtype)}", "wb") as f:
                            f.write(content)
                else:
                    
                    saved_list.append(False)
                    logging.warning(f"No saved: {path_file}, because content is None")
    
    # save metadata
    metadata_df = pd.DataFrame({'date': dates_list, 'task': tasks_list, 'username': users_list, 'result': results_list, 'submission': subs_list, 'lang': langs_list, 'type': types_list, 'saved': saved_list})
    metadata_df.to_csv(f"{base_path}/{course_id}.csv")

In [ ]:
def names_of_only_csv(files):
    """
    return the names of only csv files
    files: list of filenames with extension
    """
    return (
        list(
            # solo devolvemos el nombre del archivo
            map(lambda p: p[0],
                # filtramos los que terminan en .csv
                filter(lambda p: p[1] == 'csv' if len(p) == 2 else False,
                       # partimos el nombre del archivo y la extensión
                       map(lambda n: n.split('.'), files)))))

In [ ]:

def merge_all_metada(base_path, save=True):
    partial_metadata_list = []
    files = os.listdir(base_path)
    names = names_of_only_csv(files)
    for courseid in names:
        partial_df = pd.read_csv(f"{base_path}/{courseid}.csv") 
        partial_df['courseid'] = courseid
        partial_metadata_list.append(partial_df)
        
    metadata_df = pd.concat(partial_metadata_list)
    metadata_df = metadata_df[['date', 'courseid', 'task', 'username', 'result', 'submission', 'lang', 'type', 'saved']]
    if save:
        metadata_df.to_csv(f"{base_path}/metadata.csv", index=False)
    return metadata_df

In [ ]:
def save_submissions_by_course_wrapper(base_path, limit=0):
    def f(course_id):
        print(f"Download info for course: {course_id}")
        save_submissions_by_course(base_path=base_path, course_id=course_id, limit=limit)
    return f

In [ ]:
os.makedirs('dataset_project_ml')

In [ ]:
save_subs_configured = save_submissions_by_course_wrapper(base_path='dataset_project_ml')

In [ ]:
# serial
for course in all_courses_df['courseid'].values:
    save_subs_configured(course)

Download info for course: IALPCP-GroupMLDS-2021-2


/tmp/ipykernel_1096/20138679.py:26: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  failed_subs = fs_submissions_col.find({'taskid': task['task_id'], 'username': [ut['username']], 'status': 'done', 'result': 'failed'}).count()


Download info for course: IALPCP2-GroupMLDS-2021-2


In [ ]:
# merge all csv
metadata_df = merge_all_metada(base_path='dataset_project_ml', save=True)

In [ ]:
metadata_df.shape

(16507, 9)

In [ ]:
metadata_df.head()

,date,courseid,task,username,result,submission,lang,type,saved
0,2021-09-22 12:22:11.021,IALPCP-GroupMLDS-2021-2,example_1,1ca3b9559f8c1785fa68f9c881bad75d,failed,614b664314f5fb141c9d4570,python3,code_multiple_languages,True
1,2021-09-22 12:22:33.940,IALPCP-GroupMLDS-2021-2,example_1,1ca3b9559f8c1785fa68f9c881bad75d,success,614b665914f5fb141c9d4577,python3,code_multiple_languages,True
2,2021-09-01 11:23:36.585,IALPCP-GroupMLDS-2021-2,example_1,25654364f052d7175e182db76aa28a7a,failed,612fa908470407b924c686f5,python3,code_multiple_languages,True
3,2021-09-01 11:25:27.472,IALPCP-GroupMLDS-2021-2,example_1,25654364f052d7175e182db76aa28a7a,failed,612fa977470407b924c68701,python3,code_multiple_languages,True
4,2021-09-01 11:26:00.128,IALPCP-GroupMLDS-2021-2,example_1,25654364f052d7175e182db76aa28a7a,failed,612fa998470407b924c68707,python3,code_multiple_languages,True


In [ ]:
aux = metadata_df[['task', 'username', 'result', 'submission']].groupby(['task', 'username', 'result']).count()
aux

submission
task             username                         result             
demo2_notebooks  ade0fd9e2b7ca9e98600c133ff1feed2 failed            2
                                                  success           1
ex1_bank         191411e69b74969288c76b67d7be9eb0 failed            4
                                                  success           1
                 6cfc494c101ae14aae7542913ded3ce8 failed            2
...                                                               ...
p9_string_slices f0aa19721aff81b2c5d3da2ccd10c3a1 success           1
                 f378e4b762177284f73df4c64bb0c01b failed            2
                                                  success           1
                 fa55d3d687eb284094612392f6c59284 failed            1
                                                  success           2

[6668 rows x 1 columns]

In [ ]:
metadata_df[['lang', 'saved']].groupby('lang').count()

,saved
lang,
python,328
python3,16179


In [ ]:
#delete all python submissions, keep only python3
to_delete_df = metadata_df[metadata_df['lang'] == 'python'][['task', 'username']]
to_delete_df['path'] = ('dataset_project_ml/' +
                        to_delete_df['task'])

In [ ]:
import shutil

In [ ]:
for td in to_delete_df['path'].unique():
    print(td)
    shutil.rmtree(td)

dataset_project_ml/demo2_notebooks
dataset_project_ml/ex1_bank
dataset_project_ml/ex2_pandas
dataset_project_ml/h1_oop
dataset_project_ml/h2_pandas
dataset_project_ml/p15_functions
dataset_project_ml/p16_functional_programming
dataset_project_ml/p17_read_files
dataset_project_ml/p18_modules_pandas


In [ ]:
metadata_df = metadata_df[metadata_df['lang'] == 'python3']
metadata_df.to_csv('dataset_project_ml/metadata.csv')

In [ ]:
metadata_df.shape

(16179, 9)